In [1]:
import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

In [2]:
app = Flask(__name__)

In [3]:
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///db/bellybutton.sqlite"
db = SQLAlchemy(app)

C:\Programas\Anaconda3\envs\PythonData\lib\site-packages\flask_sqlalchemy\__init__.py:835: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [4]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(db.engine, reflect=True)

In [5]:
# Save references to each table
Samples_Metadata = Base.classes.sample_metadata
Samples = Base.classes.samples

In [21]:
    #"""Return a list of sample names."""
# Use Pandas to perform the sql query
stmt = db.session.query(Samples).statement
df = pd.read_sql_query(stmt, db.session.bind)

In [22]:
df.head()

,otu_id,otu_label,940,941,943,944,945,946,947,948,...,1562,1563,1564,1572,1573,1574,1576,1577,1581,1601
0,1,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,3,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,4,Archaea;Euryarchaeota;Methanobacteria;Methanob...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,5,Archaea;Euryarchaeota;Methanobacteria;Methanob...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
sample = "940"

In [29]:
sample_data = df.loc[df[sample] > 1, ["otu_id", "otu_label", sample]]
sample_data.head()

,otu_id,otu_label,940
40,41,Bacteria,71
120,121,Bacteria,2
158,159,Bacteria,2
164,165,Bacteria,12
169,170,Bacteria,2


In [30]:
sample_data.sort_values(by=sample, ascending=False, inplace=True)
sample_data.head()

,otu_id,otu_label,940
1166,1167,Bacteria;Bacteroidetes;Bacteroidia;Bacteroidal...,163
2858,2859,Bacteria;Firmicutes;Clostridia;Clostridiales;I...,126
481,482,Bacteria,113
2263,2264,Bacteria;Firmicutes;Clostridia;Clostridiales;I...,78
40,41,Bacteria,71


In [31]:
 data = {
        "otu_ids": sample_data.otu_id.values.tolist(),
        "sample_values": sample_data[sample].values.tolist(),
        "otu_labels": sample_data.otu_label.tolist(),
    }

In [32]:
print(data)

{'otu_ids': [1167, 2859, 482, 2264, 41, 1189, 352, 189, 2318, 1977, 3450, 874, 1959, 2191, 1950, 2077, 2275, 944, 2184, 2244, 2024, 2419, 2811, 165, 2782, 2247, 2011, 2396, 830, 2964, 1795, 2722, 307, 2178, 2908, 1193, 2167, 1208, 2039, 1274, 2739, 2737, 1314, 1962, 2186, 2335, 2936, 907, 833, 2483, 2475, 2491, 2291, 159, 2571, 2350, 2342, 2546, 725, 170, 1505, 513, 259, 1169, 258, 1232, 1497, 1498, 1503, 412, 2235, 1960, 1968, 121, 2065, 340, 2110, 2188, 357, 342], 'sample_values': [163, 126, 113, 78, 71, 51, 50, 47, 40, 40, 37, 36, 30, 28, 25, 23, 22, 19, 19, 14, 13, 13, 13, 12, 12, 11, 11, 11, 10, 10, 10, 8, 7, 7, 7, 6, 5, 5, 5, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], 'otu_labels': ['Bacteria;Bacteroidetes;Bacteroidia;Bacteroidales;Porphyromonadaceae;Porphyromonas', 'Bacteria;Firmicutes;Clostridia;Clostridiales;IncertaeSedisXI;Peptoniphilus', 'Bacteria', 'Bacteria;Firmicutes;Clostridia;Clostridiales;

In [35]:
def samples(sample):
    """Return `otu_ids`, `otu_labels`,and `sample_values`."""
    stmt = db.session.query(Samples).statement
    df = pd.read_sql_query(stmt, db.session.bind)

    # Filter the data based on the sample number and
    # only keep rows with values above 1
    sample_data = df.loc[df[sample] > 1, ["otu_id", "otu_label", sample]]

    # Sort by sample
    sample_data.sort_values(by=sample, ascending=False, inplace=True)

    # Format the data to send as json
    data = {
        "otu_ids": sample_data.otu_id.values.tolist(),
        "sample_values": sample_data[sample].values.tolist(),
        "otu_labels": sample_data.otu_label.tolist(),
    }
    return jsonify(data)


In [37]:
samples("940")

RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
to interface with the current application object in some way. To solve
this, set up an application context with app.app_context().  See the
documentation for more information.

In [9]:
var = (list(df.columns)[2:])

In [10]:
print(var)

['940', '941', '943', '944', '945', '946', '947', '948', '949', '950', '952', '953', '954', '955', '956', '958', '959', '960', '961', '962', '963', '964', '966', '967', '968', '969', '970', '971', '972', '973', '974', '975', '978', '1233', '1234', '1235', '1236', '1237', '1238', '1242', '1243', '1246', '1253', '1254', '1258', '1259', '1260', '1264', '1265', '1273', '1275', '1276', '1277', '1278', '1279', '1280', '1281', '1282', '1283', '1284', '1285', '1286', '1287', '1288', '1289', '1290', '1291', '1292', '1293', '1294', '1295', '1296', '1297', '1298', '1308', '1309', '1310', '1374', '1415', '1439', '1441', '1443', '1486', '1487', '1489', '1490', '1491', '1494', '1495', '1497', '1499', '1500', '1501', '1502', '1503', '1504', '1505', '1506', '1507', '1508', '1510', '1511', '1512', '1513', '1514', '1515', '1516', '1517', '1518', '1519', '1521', '1524', '1526', '1527', '1530', '1531', '1532', '1533', '1534', '1535', '1536', '1537', '1539', '1540', '1541', '1542', '1543', '1544', '1545', 

In [ ]:
###############

In [12]:
    sel = [
        Samples_Metadata.sample,
        Samples_Metadata.ETHNICITY,
        Samples_Metadata.GENDER,
        Samples_Metadata.AGE,
        Samples_Metadata.LOCATION,
        Samples_Metadata.BBTYPE,
        Samples_Metadata.WFREQ,
    ]

In [13]:
results = db.session.query(*sel).filter(Samples_Metadata.sample == 940).all()

In [14]:
print(results)

[(940, 'Caucasian', 'F', 24.0, 'Beaufort/NC', 'I', 2.0)]


In [18]:
sample_metadata = {}
for result in results:
        sample_metadata["sample"] = result[0]
        sample_metadata["ETHNICITY"] = result[1]
        sample_metadata["Gender"] = result[2]
        sample_metadata["AGE"] = result[3]
        sample_metadata["LOCATION"] = result[4]
        sample_metadata["BBTYPE"] = result[5]
        sample_metadata["WFREQ"] = result[6]

print(sample_metadata)

{'sample': 940, 'ETHNICITY': 'Caucasian', 'Gender': 'F', 'AGE': 24.0, 'LOCATION': 'Beaufort/NC', 'BBTYPE': 'I', 'WFREQ': 2.0}


In [20]:
stmt = db.session.query(Samples).statement
df_smt = pd.read_sql_query(stmt, db.session.bind)

IndentationError: unexpected indent (<ipython-input-20-34183479eed1>, line 2)

In [23]:
    stmt = db.session.query(Samples).statement
    df = pd.read_sql_query(stmt, db.session.bind)

In [26]:
df.head(20)

,otu_id,otu_label,940,941,943,944,945,946,947,948,...,1562,1563,1564,1572,1573,1574,1576,1577,1581,1601
0,1,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,3,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,4,Archaea;Euryarchaeota;Methanobacteria;Methanob...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,5,Archaea;Euryarchaeota;Methanobacteria;Methanob...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
5,6,Archaea;Euryarchaeota;Methanobacteria;Methanob...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
6,7,Bacteria,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
7,8,Bacteria,0,0,0,0,0,0,0,0.0,...,0,0,1,0,0,0,0,0,0,0
8,9,Bacteria,0,0,0,0,0,1,0,0.0,...,0,0,0,0,0,0,0,0,0,0
9,10,Bacteria,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
